# Grounding in LTN (continuation)

This tutorial explains how to ground connectives and quantifiers in LTN. It expects some familiarity with the first tutorial on grounding non-logical symbols (constants, variables, functions and predicates).

In [1]:
import ltn
import numpy as np
import torch

## Connectives

LTN supports various logical connectives. They are grounded using fuzzy semantics. The implementation of some of the most
common fuzzy semantics using PyTorch primitives is included in the `ltn.fuzzy_ops` module.

In general, we recommend using the following semantics for LTN connectives:
* the standard negation  $\lnot u = 1-u$;
* the product t-norm $u \land v = uv$;
* the product t-conorm (probabilistic sum) $u \lor v = u+v-uv$;
* the Reichenbach implication $u \rightarrow v = 1 - u + uv$;

where $u$ and $v$ denote two truth values in $[0,1]$.

For more details about the choice of the right fuzzy semantics for your task, we suggest reading the complementary notebook (2b-grounding_connectives.ipynb).

In LTN, creating a connective is really simple. It is possible to use the constructor `Connective()`, which takes as input a
unary or binary fuzzy connective semantics. It is possible to choice the semantics from the `ltn.fuzzy_ops` module.

In this tutorial, we use the connective fuzzy semantics suggested above.

In [2]:
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())

In particular, the wrapper `ltn.Connective` allows to use the operators with LTN formulas. Specifically, it takes care
of combining sub-formulas which have different variables appearing in them (the sub-formulas may have different dimensions
that need to be "broadcasted" in order to apply the connective).

In this example, we create two variables with different number of individuals, two constants, and a predicate measuring the similarity between two points in $\mathbb{R}^2$.

In [3]:
x = ltn.Variable('x', torch.randn((10, 2))) # 10 values in R²
y = ltn.Variable('y', torch.randn((5, 2))) # 5 values in R²

c1 = ltn.Constant(torch.tensor([0.5, 0.0]))
c2 = ltn.Constant(torch.tensor([4.0, 2.0]))

Eq = ltn.Predicate(func=lambda x, y: torch.exp(-torch.norm(x - y, dim=1))) # predicate measuring similarity

Eq(c1, c2).value

tensor(0.0178)

Now, we check the behavior of the logical connectives. It is easy to see that they behave according to the definition
written at the beginning of the tutorial.

Notice the shape printed in the last two lines of code. In the first one, since only variable x appears in the formula,
we have a shape of 5. The formula has been evaluated for each x.
In the second one, since both x and y appear in the formula, we have a shape of 10x5.
The formula has been evaluated for each combination of x and y.

In [4]:
Not(Eq(c1, c2)).value

tensor(0.9822)

In [5]:
Implies(Eq(c1, c2), Eq(c2, c1)).value

tensor(0.9825)

In [6]:
# Notice the dimension of the outcome: the result is evaluated for every x.
And(Eq(x, c1), Eq(x, c2)).shape()

torch.Size([10])

In [7]:
# Notice the dimensions of the outcome: the result is evaluated for every x and y.
# Notice also that y did not appear in the 1st argument of `Or`;
# the connective broadcasts the results of its two arguments to match.
Or(Eq(x, c1), Eq(x, y)).shape()

torch.Size([10, 5])

Notice the access to the `value` attribute or `shape()` method of the evaluation of the connective. This is done because LTN connectives return `LTNObject` instances, like it happens for predicates and functions.

## Quantifiers

LTN suppports universal and existential quantification. They are grounded using aggregation operators. We recommend using the following two operators:

- existential quantification ("exists"):
the generalized mean (`pMean`) $\mathrm{pM}(u_1,\dots,u_n) = \biggl( \frac{1}{n} \sum\limits_{i=1}^n u_i^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$;
- universal quantification ("for all"):
the generalized mean of "the deviations w.r.t. the truth" (`pMeanError`) $\mathrm{pME}(u_1,\dots,u_n) = 1 - \biggl( \frac{1}{n} \sum\limits_{i=1}^n (1-u_i)^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$;

where $u_1,\dots,u_n$ is a list of truth values in $[0,1]$.

In LTN, creating a quantifier is really simple. It is possible to use the constructor `Quantifier()`, which takes as input
an aggregation semantics and a character indicating which type of quantification is associated to the quantifier ("e" for exists, "f" for forall).

In this example, we create the quantifiers using the fuzzy semantics proposed above.

In [8]:
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")

The wrapper `ltn.Quantifier` allows to use the aggregators with LTN formulas. It takes care of selecting the tensor (formula)
dimensions to aggregate, given some variables in arguments.

In this example, we create variables and predicate similar to the previous example on connectives.

In [9]:
x = ltn.Variable('x', torch.randn((10, 2))) # 10 values in R²
y = ltn.Variable('y', torch.randn((5, 2))) # 5 values in R²

Eq = ltn.Predicate(func=lambda x, y: torch.exp(-torch.norm(x - y, dim=1))) # predicate measuring similarity

Eq(x, y).shape()

torch.Size([10, 5])

Now, we apply some quantifiers to the formula, and we see how this effects the output and its shape.

In the first case, we have quantified on x, so the output has shape 5. This means we have removed the dimension related to x and only dimension related to y is left. The shape is 5 since y has 5 individuals.

In the other three cases, the output is a scalar since the quantification has been performed on both variables. This means the aggregation has been performed on both dimension, namely the dimension of x and the dimension of y.

In [10]:
Forall(x, Eq(x, y)).shape()

torch.Size([5])

In [11]:
Forall([x, y], Eq(x, y)).value

tensor(0.3004)

In [12]:
Exists([x, y], Eq(x, y)).value

tensor(0.4113)

In [13]:
Forall(x, Exists(y, Eq(x, y))).value

tensor(0.3635)

Notice the access to the `value` attribute or `shape()` method of the evaluation of the quantifier. This is done because LTN quantifiers return `LTNObject` instances, like it happens for predicates, functions, and connectives.

It is important to observe that when the quantification is performed on a single variable, it is possible to give the variable
to the quantifier as it comes, instead, if the quantification has to be performed on more variables, it is required to give the variables to the quantifier
through a list, as it happens for the second and third example.

## Semantics for quantifiers

`pMean` semantics can be understood as a soft-maximum that depends on the hyper-paramer $p$:
- $p \to 1$: the operator tends to `mean`,
- $p \to +\infty$: the operator tends to `max`.

Similarly, `pMeanError` semantics can be understood as a soft-minimum:
- $p \to 1$: the operator tends to `mean`,
- $p \to +\infty$: the operator tends to `min`.

Intuitively, $p$ offers flexibility in writing more or less strict formulas, to account for outliers in the data depending on the application.

Note that different choices of $p$ could have strong implications during the training (see 2b-grounding_connectives.ipynb).

One can set a default value for $p$ when initializing the operator, or can use different values at each call of the operator.

In this example, we use the quantifiers with different values of the $p$ parameter.

In general, higher is $p$ and easier is the existential quantification to be satisfied, while harder is the universal quantification to be satisfied.

In [14]:
Forall(x, Eq(x, c1), p=2).value

tensor(0.4100)

In [15]:
Forall(x, Eq(x, c1), p=10).value

tensor(0.2308)

In [16]:
Exists(x, Eq(x, c1), p=2).value

tensor(0.4980)

In [17]:
Exists(x, Eq(x, c1), p=10).value

tensor(0.6306)

## Diagonal Quantification

Given 2 (or more) variables, there are scenarios where one wants to express statements about specific pairs (or tuples) only, such that the $i$-th tuple contains the $i$-th instances of the variables.
In other words, in some cases we do not want to evaluate a formula on all the possible combination of individuals of the variables involved in it.

We allow this using `ltn.diag` (diagonal quantification).

**Note**: diagonal quantification assumes that the variables have the same number of individuals.
This is intuitive since we need a one-to-one correspondence between the individuals of the variables involved in the quantification.

In simplified pseudo-code, the usual quantification would compute:
```python
for x_i in x:
    for y_j in y:
        results.append(P(x_i,y_j))
aggregate(results)
```
In contrast, diagonal quantification would compute:
```python
for x_i, y_i in zip(x,y):
    results.append(P(x_i,y_i))
aggregate(results)
```

We illustrate `ltn.diag` in the following setting:
- the variable $x$ denotes 100 individuals in $\mathbb{R}^{2\times2}$,
- the variable $l$ denotes 100 one-hot labels in $\mathbb{N}^3$ (3 possible classes),
- $l$ is grounded according to $x$ such that each pair $(x_i,l_i)$, for $i=0..100$ denotes one correct example from the dataset,
- the classifier $C(x,l)$ gives a confidence value in $[0,1]$ of the sample $x$ corresponding to the label $l$.

In [18]:
# The values are generated at random, for the sake of illustration.
# In a real scenario, they would come from a dataset.
samples = torch.randn((100, 2, 2)) # 100 R^{2x2} values
labels = torch.randint(0, 3, size=(100,)) # 100 labels (class 0/1/2) that correspond to each sample
onehot_labels = torch.nn.functional.one_hot(labels, num_classes=3)

x = ltn.Variable("x", samples)
l = ltn.Variable("l", onehot_labels)

class ModelC(torch.nn.Module):
    def __init__(self):
        super(ModelC, self).__init__()
        self.elu = torch.nn.ELU()
        self.softmax = torch.nn.Softmax(dim=1)
        self.dense1 = torch.nn.Linear(4, 5)
        self.dense2 = torch.nn.Linear(5, 3)

    def forward(self, x, l):
        x = torch.flatten(x, start_dim=1)
        x = self.elu(self.dense1(x))
        x = self.softmax(self.dense2(x))
        return torch.sum(x * l, dim=1)

C = ltn.Predicate(ModelC())

If some variables are marked using `ltn.diag`, LTN will only compute their "zipped" results (instead of the usual "broadcasting").

In other words, the formula will be evaluated only on specific tuples of individuals instead of being evaluated on all the possible combinations of individuals of the variables.

It is possible to observe that the shape of the first evaluation is 100x100. This happens because LTN generates all the possible
combinations of individuals of $x$ and $l$, and then applies the predicate.

Note how the shape changes after applying `ltn.diag`. This happens because the individuals of the two variables are taken in one-to-one correspondence and the predicate
is computed only on that specific tuples of individuals.

Notice how the `free_vars` attribute of the variables is changed after a diagonal setting has been set. It is possible to recognize
if a variable is in diagonal setting by checking if its label begins with "diag_". In particular, all the variables in the same
diagonal setting will share the same label.

It is possible to use `ltn.undiag` to obtain the opposite behavior. In other words, it removes the diagonal setting from the variables
and restore the usual LTN broadcasting for the variables. This is clarified from the last print.

In [19]:
print(C(x, l).shape()) # Computes the 100x100 combinations
ltn.diag(x, l) # sets the diag behavior for x and l
print(C(x, l).shape())# Computes the 100 zipped combinations
print(x.free_vars)
print(l.free_vars)
ltn.undiag(x, l) # resets the normal behavior
print(C(x, l).shape()) # Computes the 100x100 combinations

torch.Size([100, 100])
torch.Size([100])
['diag_x_l']
['diag_x_l']
torch.Size([100, 100])


In practice, `ltn.diag` is designed to be used with quantifiers.
Every quantifier automatically calls `ltn.undiag` after the aggregation is performed, so that the variables keep their normal
behavior outside of the formula.
Therefore, it is recommended to use `ltn.diag` only in quantified formulas as follows.

Notice the call to `ltn.undiag` by the quantifier. After the quantification has been performed, the two variables restore their original labels.

In [20]:
x, l = ltn.diag(x, l)
print(x.free_vars)
print(l.free_vars)
print(Forall([x, l], C(x, l)).value) # Aggregates only on the 100 "zipped" pairs.
                                    # Automatically calls `ltn.undiag` so the behavior of x/l is unchanged outside of this formula.
print(x.free_vars)
print(l.free_vars)

['diag_x_l']
['diag_x_l']
tensor(0.3382, grad_fn=<RsubBackward1>)
['x']
['l']


## Guarded Quantifiers

One may wish to quantify over a set of elements whose grounding satisfy some **boolean** condition.

Let us assume $x$ is a variable from some domain and $m$ is a mask function that returns boolean values (that is, $0$ or $1$, not continuous truth degrees in $[0,1]$) for each element of the domain.

In guarded quantification, one has quantifications of the form:

- $(\forall x: m(x)) \text{ } \phi(x)$
which means "every x satisfying $m(x)$ also satisfies $\phi(x)$";
- $(\exists x: m(x)) \text{ } \phi(x)$
which means "some x satisfying $m(x)$ also satisfies $\phi(x)$".

The mask $m$ can also depend on other variables in the formula. For instance, the quantification $\exists y (\forall x:m(x,y)) \text{ } \phi(x,y)$ is also a valid sentence.

Let us consider the following example, that states that there exists an euclidian distance $d$ below which all pairs of points $x$, $y$ should be considered as similar:
$\exists d \ (\forall x,y : \mathrm{dist}(x,y) < d) \ ( \mathrm{Eq}(x,y))) $.

In this example, $Eq$ is a predicate measuring the similarity between two points, while $dist$ is a function which computes the euclidean distance between two points.

In [21]:
Eq = ltn.Predicate(func=lambda x, y: torch.exp(-torch.norm(x - y, dim=1))) # predicate measuring similarity

points = torch.rand((50, 2)) # 50 values in [0,1]^2
x = ltn.Variable("x", points)
y = ltn.Variable("y", points)
d = ltn.Variable("d", torch.tensor([.1,.2,.3,.4,.5,.6,.7,.8,.9]))

In [24]:
dist = lambda x, y: torch.unsqueeze(torch.norm(x.value - y.value, dim=1), 1) # function measuring euclidian distance
Exists(d,
      Forall([x, y],
            Eq(x, y),
            cond_vars=[x, y, d],
            cond_fn=lambda x, y, d: dist(x, y) < d.value
            )).value

tensor(0.7583, dtype=torch.float64)

As showed in the example, in order to use a guarded quantification it is enough to specify some condition variables (`cond_vars` parameter) and a
condition function (`cond_fn` parameter). In particular, `cond_vars` requires a list of LTN variables, while
`cond_fn` requires a function. The function computes a boolean mask, then the mask is used to select the values on which
the aggregation has to be computed.

In this specific example, the guarded quantification is used for aggregating over pairs of points whose distance is under a certain
threshold, specified by variable $d$. All the other pairs of points are not considered during the aggregation.

The guarded option is particularly useful to propagate gradients (see notebook on learning) only over a subset of the
domains, namely the domains which verifies the condition $m$.